In [1]:
import pandas as pd
import re

In [2]:
fairy_tales = pd.read_csv("../data/fairy_tales.csv")

chunks = []
sentences_per_chunk = 3
overlap_sentences = 1

for title, text in zip(fairy_tales["title"], fairy_tales["text"]):
    sentences = [s.strip().lower() for s in re.split(r"(?<=[.!?])\s+", text) if s.strip()]

    i = 0
    while i < len(sentences):
        chunk = " ".join(sentences[i:i + sentences_per_chunk])
        chunks.append(f"[{title.lower()}] {chunk}")
        i += sentences_per_chunk - overlap_sentences

print(f"chunks: {len(chunks)}")

chunks: 454


In [37]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [38]:
preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")

preprocessed = preprocessor(chunks)
embeddings = encoder(preprocessed)["default"].numpy()

norms = np.linalg.norm(embeddings, 2, axis=1, keepdims=True)
embeddings = embeddings / norms

print(f"shape: {embeddings.shape}")

shape: (454, 768)


#### USE-QA

In [70]:
def answer_question(question, chunks, embeddings):
    question_preprocessed = preprocessor([question])    
    question_embedding = encoder(question_preprocessed)["default"].numpy()
    question_norm = question_embedding / np.linalg.norm(question_embedding, 2, axis=1, keepdims=True)

    similarities = np.matmul(question_norm, embeddings.T)[0]

    most_similar_idx = similarities.argmax()
    return chunks[most_similar_idx], similarities[most_similar_idx]

In [71]:
question = "Мысық нені төгеді?"
chunk, similarity = answer_question(question, chunks, embeddings)

print(f"Relevance: {similarity:.4f}")
print(f"Chunk: {chunk}")

Relevance: 0.4818
Chunk: [мақта қыз бен мысық] – дейді.мысық тауыққа барады.– тауық, тауық, маған жұмыртқа берші! – дейді.тауықтар:– бізге дән әкеліп берсең, біз саған жұмыртқа береміз, – дейді.мысық: «енді не қыламын?» – деп бара жатса, бір ін қазып жатқан тышқанды көреді. мысық тышқанды бас салады.– жаныңның барында айт!


In [72]:
question = "Дүкенші мысықтан не сұрайды?"
chunk, similarity = answer_question(question, chunks, embeddings)

print(f"Relevance: {similarity:.4f}")
print(f"Chunk: {chunk}")

Relevance: 0.4666
Chunk: [мақта қыз бен мысық] – деп, мейізді өзі жеп қояды. жеп болған соң мысық келіп:– неге шақырдың? – деп сұрайды.мақта қыз айтпайды.


#### Top-3

In [64]:
def answer_question(question, chunks, embeddings):
    question_preprocessed = preprocessor([question])    
    question_embedding = encoder(question_preprocessed)["default"].numpy()
    question_norm = question_embedding / np.linalg.norm(question_embedding, 2, axis=1, keepdims=True)

    similarities = np.matmul(question_norm, embeddings.T)[0]

    most_similar_idx = similarities.argsort()[-3:][::-1]
    return [(chunks[i], similarities[i]) for i in most_similar_idx]

In [65]:
question = "Мысық нені төгеді?"
results = answer_question(question, chunks, embeddings)

for chunk, similarity in results:
    print(f"Relevance: {similarity:.4f} \nChunk: {chunk}\n")

Relevance: 0.4818 
Chunk: [мақта қыз бен мысық] – дейді.мысық тауыққа барады.– тауық, тауық, маған жұмыртқа берші! – дейді.тауықтар:– бізге дән әкеліп берсең, біз саған жұмыртқа береміз, – дейді.мысық: «енді не қыламын?» – деп бара жатса, бір ін қазып жатқан тышқанды көреді. мысық тышқанды бас салады.– жаныңның барында айт!

Relevance: 0.4257 
Chunk: [мақта қыз бен мысық] – деп, мейізді өзі жеп қояды. жеп болған соң мысық келіп:– неге шақырдың? – деп сұрайды.мақта қыз айтпайды.

Relevance: 0.4213 
Chunk: [мақта қыз бен мысық] мысық тышқанды бас салады.– жаныңның барында айт! үйіңде не бар? – дейді.тышқан қорыққанынан:– үйімде бір табақ тары бар, – дейді.мысық:– маған бір уыс тары бер, – дейді.тышқан үйіне барып, бір уыс тары әкеліп береді; мысық тарыны апарып тауыққа береді, тауық жұмыртқа береді; жұмыртқаны апарып дүкеншіге береді; дүкенші сағыз береді; сағызды апарып қыздарға береді, қыздар су береді; суды апарып ағашқа береді; ағаш жапырақ береді; жапырақты апарып сиырға береді, сиы

In [66]:
question = "Дүкенші мысықтан не сұрайды?"
results = answer_question(question, chunks, embeddings)

for chunk, similarity in results:
    print(f"Relevance: {similarity:.4f} \nChunk: {chunk}\n")

Relevance: 0.4666 
Chunk: [мақта қыз бен мысық] – деп, мейізді өзі жеп қояды. жеп болған соң мысық келіп:– неге шақырдың? – деп сұрайды.мақта қыз айтпайды.

Relevance: 0.4642 
Chunk: [мақта қыз бен мысық] мысық тышқанды бас салады.– жаныңның барында айт! үйіңде не бар? – дейді.тышқан қорыққанынан:– үйімде бір табақ тары бар, – дейді.мысық:– маған бір уыс тары бер, – дейді.тышқан үйіне барып, бір уыс тары әкеліп береді; мысық тарыны апарып тауыққа береді, тауық жұмыртқа береді; жұмыртқаны апарып дүкеншіге береді; дүкенші сағыз береді; сағызды апарып қыздарға береді, қыздар су береді; суды апарып ағашқа береді; ағаш жапырақ береді; жапырақты апарып сиырға береді, сиыр қатық береді; қатықты апарып мақта қызға береді, мақта қыз мысықтың құйрығын береді.

Relevance: 0.4474 
Chunk: [жануарлар патшасы кім?] бірақ менің бір ұсынысым бар: патша өзімен бірге барлық жануарларды сүйе білуі керек. күштілерге қарамастан, кішігірім жануарларды да қорғай алатын, ақылды әрі мейірімді патша болғаны жөн.

#### TF-IDF

In [67]:
def answer_question(question, chunks, embeddings):
    question_preprocessed = preprocessor([question])    
    question_embedding = encoder(question_preprocessed)["default"].numpy()
    question_norm = question_embedding / np.linalg.norm(question_embedding, 2, axis=1, keepdims=True)

    similarities = np.matmul(question_norm, embeddings.T)[0]

    top_idx = similarities.argsort()[-3:][::-1]
    return top_idx, similarities[top_idx]

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
question = "Мысық нені төгеді?"
relevant_chunks, similarities = answer_question(question, chunks, embeddings)

vectorizer = TfidfVectorizer()
texts = [question] + [chunks[i] for i in relevant_chunks]
tfidf_matrix = vectorizer.fit_transform(texts)

tfidf_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

combined = 0.4 * similarities + 0.6 * tfidf_scores
most_relevant_chunk = relevant_chunks[combined.argmax()]

print(chunks[most_relevant_chunk])

[мақта қыз бен мысық] – дейді.мысық тауыққа барады.– тауық, тауық, маған жұмыртқа берші! – дейді.тауықтар:– бізге дән әкеліп берсең, біз саған жұмыртқа береміз, – дейді.мысық: «енді не қыламын?» – деп бара жатса, бір ін қазып жатқан тышқанды көреді. мысық тышқанды бас салады.– жаныңның барында айт!


In [56]:
question = "Дүкенші мысықтан не сұрайды?"
relevant_chunks, similarities = answer_question(question, chunks, embeddings)

vectorizer = TfidfVectorizer()
texts = [question] + [chunks[i] for i in relevant_chunks]
tfidf_matrix = vectorizer.fit_transform(texts)

tfidf_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

combined = 0.4 * similarities + 0.6 * tfidf_scores
most_relevant_chunk = relevant_chunks[combined.argmax()]

print(chunks[most_relevant_chunk])

[мақта қыз бен мысық] – деп, мейізді өзі жеп қояды. жеп болған соң мысық келіп:– неге шақырдың? – деп сұрайды.мақта қыз айтпайды.
